In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [2]:
#RICE
path_genome = '/home/juan/Desktop/juan/bio/data/genomes/rice/IRGSP-1.0_genome.fasta'
path_gff = 'data/rice/allmites.fasta.filtered.csv.gff'
path_out_dir = 'data/rice/by_family/'

In [ ]:
path_genome = '/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa'
path_gff = 'data/wheat/allmites.fasta.filtered.csv.gff'
path_out_dir = 'data/wheat/by_family/'

In [3]:
df = pd.read_csv(path_gff, sep='\t')

In [4]:
df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
print(len(df.index))
df.head(2)

588179


,seqname,source,feature,start,end,score,strand,frame,attribute
0,chr09,MITE_Tracker,TE,13335108,13334681,.,+,.,MITE_T_1|chr09|9556469|9556924|TGCA|473|F1
1,chr09,MITE_Tracker,TE,7036771,7036335,.,+,.,MITE_T_1|chr09|9556469|9556924|TGCA|473|F1


In [5]:
fasta_seq = SeqIO.parse(path_genome, 'fasta')
buffer_mites = {}

In [ ]:
for record in fasta_seq:
    dff_extract = df[df.seqname == record.id]
    count = 0
    curr = 0
    total = len(dff_extract.index)
    print(record.id, len(dff_extract.index))
    clean_seq = ''.join(str(record.seq).splitlines())
    for k,v in dff_extract.iterrows():
        start = min(v.start,v.end)
        end = max(v.start,v.end)
        new_seq = clean_seq[start:end]
        att = v.attribute
        id = att + "_" + record.id + "_" + str(start) + '_' + str(end)
        seq = SeqRecord(Seq(new_seq), id=id, description="_")
        if not att in buffer_mites:
            buffer_mites[att] = []
        buffer_mites[att].append(seq)
        count += 1
        curr_new = int(count * 100 * 1.0 / (total * 1.0))
        if curr_new != curr:
            curr = curr_new
            if curr_new % 1 == 0:
                print(curr_new)

In [7]:
for k,v in buffer_mites.items():
    SeqIO.write(v, path_out_dir + k + ".fasta", "fasta")